In [171]:
## --- 2 SATELLITES DIFFERENTIAL DRAG ALGORITHM ---

from astropy import units as u

from poliastro.bodies import Earth
from poliastro.constants import rho0_earth, H0_earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import CowellPropagator
from poliastro.twobody.elements import mean_motion
from poliastro.core.perturbations import J2_perturbation, atmospheric_drag_exponential
from poliastro.core.propagation import func_twobody 
from poliastro.util import Time

from atmo_drag_functions import jb2008_pertubation
from pyatmos import download_sw_jb2008,read_sw_jb2008


from numba import njit as jit

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

print('--- DIFFERENTIAL DRAG CONTROL - TWO SATELLITES SCENARIO ---')

## 1 - INITIALIZATION

# initial orbital elements of FLOCK 1C-1 (40027) and FLOCK 1C-2 (40029) from TLEs
# Constellation parameters
T = 2

# Assumptions: same spacecrafts, same orbital plane
# CHASER (1):
a_1, ecc_1, inc_1, raan_1, argp_1, nu_1 = 6995 * u.km, 0.001 * u.one, 97.98 * u.deg, 69 * u.deg, 275 * u.deg, 80 * u.deg

# TARGET (2):
a_2, ecc_2, inc_2, raan_2, argp_2, nu_2 = 6990 * u.km, 0.001 * u.one, 97.98 * u.deg, 69 * u.deg, 275 * u.deg, 160 * u.deg 

# Let's assign the same start date to the satellites
start_date = Time("2014-04-17 13:12:43.89", scale = "utc")

# definition of initial orbits by using poliastro
in_orbit_1 = Orbit.from_classical(Earth, a_1, ecc_1, inc_1, raan_1, argp_1, nu_1, start_date)
in_orbit_2 = Orbit.from_classical(Earth, a_2, ecc_2, inc_2, raan_2, argp_2, nu_2, start_date)

# Constants
R  = Earth.R.to(u.km).value
k  = Earth.k.to(u.km**3 / u.s**2).value
J2 = Earth.J2.value
swfile = download_sw_jb2008() 
swdata = read_sw_jb2008(swfile) 


rho0 = rho0_earth.to(u.kg/u.km**3).value
H0   = H0_earth.to(u.km).value

C_D = 2.2
LD_A_over_m = ((0.01 * u.m**2) / (1 * u.kg)).to_value(u.km**2 / u.kg)   # km**2/kg
HD_A_over_m = ((100 * u.m**2) / (1 * u.kg)).to_value(u.km**2 / u.kg)   # km**2/kg
LD_B = C_D * LD_A_over_m   # ballistic coefficient at low drag mode
HD_B = C_D * HD_A_over_m   # ballistic coefficient at high drag mode



in_state_1 = in_orbit_1.rv()
in_state_2 = in_orbit_2.rv()

--- DIFFERENTIAL DRAG CONTROL - TWO SATELLITES SCENARIO ---
The Space Weather files 'SOLFSMY.TXT' and 'DTCFILE.TXT' in C:\Users\Lorenzo/src/sw-data/ are already the latest.


In [172]:
array_1 = np.array([*in_state_1[0].value, *in_state_1[1].value])
array_2 = np.array([*in_state_2[0].value, *in_state_2[1].value])

In [180]:

#@jit
def a_d(state, R, C_D, A_over_m, epoch, swdata):
    return jb2008_pertubation(
        epoch, state, R, C_D, A_over_m, swdata
    )

HD_acc = a_d(epoch=start_date.value, state=array_1, R=R, C_D=C_D, A_over_m=HD_A_over_m, swdata=swdata)
LD_acc = a_d(epoch=start_date.value, state=array_1, R=R, C_D=C_D, A_over_m=LD_A_over_m, swdata=swdata)

print(f'{np.linalg.norm(HD_acc)}\n{np.linalg.norm(LD_acc)}')


1.966403863921548e-15
1.9664038639215482e-19


In [181]:
unit_in_1_r = in_state_1[0] / np.linalg.norm(in_state_1[0])
unit_in_1_v = in_state_1[1] / np.linalg.norm(in_state_1[1])
unit_in_2_r = in_state_2[0] / np.linalg.norm(in_state_2[0])
unit_in_2_v = in_state_2[1] / np.linalg.norm(in_state_2[1])

dot_product_r = np.dot(unit_in_2_r, unit_in_1_r)
dot_product_v = np.dot(unit_in_2_v, unit_in_1_v)

theta     = np.arccos(dot_product_r).to_value(u.deg)
theta_dot = np.arccos(dot_product_v).to_value(u.deg)

theta     = np.arccos(dot_product_r).to_value(u.deg)
theta_dot = np.arccos(dot_product_v).to_value(u.deg)

theta_dot

80.03680077140977

In [183]:
def f_LD(t0, state, k):

    du_kep = func_twobody(t0, state, k)
    ax, ay, az = a_d(state, 
                     R = R, 
                     C_D = C_D, 
                     A_over_m = LD_A_over_m,
                     epoch=start_date.value, 
                     swdata=swdata)
    du_ad = np.array([0, 0, 0, ax, ay, az])

    return du_kep + du_ad

def f_HD(t0, state, k):

    du_kep = func_twobody(t0, state, k)
    ax, ay, az = a_d(state,  
                     R = R, 
                     C_D = C_D, 
                     A_over_m = HD_A_over_m*10,
                     epoch=start_date.value,
                     swdata=swdata)
    du_ad = np.array([0, 0, 0, ax, ay, az])

    return du_kep + du_ad

LD_mode = in_orbit_1.propagate(30*u.day, method=CowellPropagator(rtol=1e-5, f=f_LD))
HD_mode = in_orbit_1.propagate(30*u.day, method=CowellPropagator(rtol=1e-5, f=f_HD))

LD_mode.n, HD_mode.n

(<Quantity 0.00108054 rad / s>, <Quantity 0.00108054 rad / s>)

In [184]:
LD_mode.a, HD_mode.a

(<Quantity 6989.04814259 km>, <Quantity 6989.04808175 km>)

In [ ]:
LD_mode.n == 

In [187]:
unit_HD_acc = HD_acc / np.linalg.norm(HD_acc)
unit_LD_acc = LD_acc / np.linalg.norm(LD_acc)

dot_product_acc = np.dot(unit_HD_acc, unit_LD_acc)
unit_HD_acc, unit_LD_acc
# theta_dot_dot   = ((np.arccos(dot_product_acc)) * u.rad).to_value(u.deg)
# theta_dot_dot

theta_dot_dot = HD_acc - LD_acc

In [88]:
theta_des     = 45
theta_dot_des = 0
delta_t_hd =  (theta_dot / theta_dot_dot)
theta_hd = 0.5 * theta_dot_dot * (delta_t_hd**2)
delta_t_wait = np.absolute((theta_hd - theta_des)) / theta_dot

print(f'delta_t_wait = {delta_t_wait}\ndelta_t_hd   = {delta_t_hd}')

delta_t_wait = nan
delta_t_hd   = nan
